# Convertendo Áudio para Texto

Então, é mais simples encontrar em qual momento tão assunto ou palavra foi dita em um arquivo de texto, ao inves de procurá-la no arquivo de áudio, utilizando a navegação do aplicativo Player de Áudio.

Para a conversão, foi necessário o uso de uma biblioteca open sourvede "Speech Recognition" (Reconhecimento de Fala). Esse tipo de biblioteca muito utilizada em ferramentas e aplicativos como Google Tradutor, e no próprio YouTube, com sua função de legendar os vídeos.

Quanto mais limpo, alto e claro o áudio for, mais "preciso" o reconhecimento da fala e conversa será.
